## MGnify notebook: retrieve info from API

In [46]:
# import libraries
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [47]:
def fetch_biomes_and_save(output_dir):
    """_summary_

    Args:
        output_dir (_type_): _description_
    """
    url = "https://www.ebi.ac.uk/metagenomics/api/v1/biomes"
    response = requests.get(url)
    if response.status_code == 200:
        biomes_data = response.json()
        biomes_list = [biome['id'] for biome in biomes_data['data']]
        
        with open(os.path.join(output_dir,"mgnify_biomes_list.txt"), 'w') as file:
            for biome_name in biomes_list:
                file.write(f"{biome_name}\n")
                
        print(f"Biomes list saved.")
        
    else:
        print("Failed to retrieve biomes")

In [48]:
def get_studies_and_analyses_summary(biome_name, experiment_type, output_dir = '../outputs'):
    """_summary_

    Args:
        biome_name (_type_): _description_
        experiment_type (_type_): _description_
        output_dir (str, optional): _description_. Defaults to '../outputs'.

    Returns:
        _type_: _description_
    """
    # set the urls
    urls = {"studies": "https://www.ebi.ac.uk/metagenomics/api/v1/studies", "analyses": "https://www.ebi.ac.uk/metagenomics/api/v1/analyses"}
    # common parts
    common_params = {"biome_name": biome_name}
    all_data = {"studies": [], "analyses": []}

    # connection request
    for key, url in urls.items():
        if key == "studies" and all_data["studies"]:
            continue

        params = common_params.copy()
        if key == "analyses":
            params.update({
                "lineage": biome_name,
                "experiment_type": experiment_type
            })

        page = 1
        
        while True:
            try:
                print(f"Retrieving data for page {page}...")
                params["page"] = page
                response = requests.get(url, params=params)
                response.raise_for_status()  # errors codes HTTP
                
                data = response.json()["data"]
                page_info = response.json()["meta"]["pagination"]
                all_data[key].extend(data)
                print(f"Page {page} retrieved successfully. Total pages: {page_info['pages']}")

                if page >= page_info["pages"]:
                    break
                page += 1
            except requests.exceptions.HTTPError as http_err:
                print(f"HTTP error occurred: {http_err} - Status code: {response.status_code}")
                break
            except Exception as err:
                print(f"An error occurred: {err}")
                break
        
        # save json files
        if key == "studies":
            output_file_path = os.path.join(output_dir, "mgnify_studies.json")
        else:
            output_file_path = os.path.join(output_dir, f"mgnify_analyses_{experiment_type}.json")
        
        with open(output_file_path, "w") as outfile:
            json.dump(all_data[key], outfile)
        print(f"{key.capitalize()} data for {experiment_type} saved to {output_file_path}")


    # building dataframes
    studies_columns = ['study_id', 'study_name', 'n_samples', 'bioproject', 'centre_name', 'biomes']
    studies_data = []
    for item in all_data['studies']:
        studies_data.append({
            'study_id': item['id'],
            'study_name': item['attributes'].get('study-name', ''),
            'n_samples': item['attributes'].get('samples-count', 0),
            'bioproject': item['attributes'].get('bioproject', ''),
            'centre_name': item['attributes'].get('centre-name', ''),
            'biomes': ", ".join([biome['id'] for biome in item['relationships']['biomes']['data']])
            })
    df_studies = pd.DataFrame(studies_data, columns=studies_columns)

    analyses_columns = ['analysis_id', 'experiment_type', 'pipeline_version', 'instrument_platform', 'study_id', 'sample_id', 'assembly_run_id']
    analyses_data = []
    for item in all_data['analyses']:
        analyses_data.append({
            'analysis_id': item['id'],
            'experiment_type': item['attributes'].get('experiment-type', ''),
            'pipeline_version': item['attributes'].get('pipeline-version', ''),
            'instrument_platform': item['attributes'].get('instrument-model', ''),
            'study_id': item['relationships']['study']['data'].get('id', '') if item['relationships'].get('study') else '',
            'sample_id': item['relationships']['sample']['data'].get('id', '') if item['relationships'].get('sample') else '',
            'assembly_run_id': item['relationships'].get('assembly', {}).get('data', {}).get('id', '') if item['attributes'].get('experiment-type') == 'assembly' else item['relationships'].get('run', {}).get('data', {}).get('id', '')
            })
    df_analyses = pd.DataFrame(analyses_data, columns=analyses_columns)

    # merging dataframe and return it
    df_summary = pd.merge(df_analyses, df_studies, on='study_id', how='left')
    
    return df_summary

In [49]:
if __name__ == "__main__":
    # setting the variables 
    biome = "root:Engineered:Wastewater"
    biome_lower = biome.replace(":", "_").lower()
    experiments = ("metagenomic","metatranscriptomic","assembly")
    output_path = '../outputs/'
    df_summary_dict = {}

    print('STARTING STEP 1: fetch_biomes_and_save')
    fetch_biomes_and_save(output_dir= output_path)

    print('STARTING STEP 2: get_studies_and_analyses_summary')
    for exp in experiments:
        print(f"Processing experiment type: {exp}")
        df_summary = get_studies_and_analyses_summary(biome_name=biome, experiment_type=exp)
        df_summary_dict[exp] = df_summary  # Aggiungi il DataFrame al dizionario

        # save the CSV file
        df_summary.to_csv(os.path.join(output_path, f"{biome_lower}_{exp}_summary.csv"), index=False)
        combined_df = pd.concat(df_summary_dict.values(), axis=0)
        combined_df.to_csv(os.path.join(output_path, 'combined_dataframe.csv'), index=False)


STARTING STEP 1: fetch_biomes_and_save
Biomes list saved.
STARTING STEP 2: get_studies_and_analyses_summary
Processing experiment type: metagenomic
Retrieving data for page 1...
Page 1 retrieved successfully. Total pages: 8
Retrieving data for page 2...
Page 2 retrieved successfully. Total pages: 8
Retrieving data for page 3...
Page 3 retrieved successfully. Total pages: 8
Retrieving data for page 4...
Page 4 retrieved successfully. Total pages: 8
Retrieving data for page 5...
Page 5 retrieved successfully. Total pages: 8
Retrieving data for page 6...
Page 6 retrieved successfully. Total pages: 8
Retrieving data for page 7...
Page 7 retrieved successfully. Total pages: 8
Retrieving data for page 8...
Page 8 retrieved successfully. Total pages: 8
Studies data for metagenomic saved to ../outputs/mgnify_studies.json
Retrieving data for page 1...
Page 1 retrieved successfully. Total pages: 40
Retrieving data for page 2...
Page 2 retrieved successfully. Total pages: 40
Retrieving data for p

## Descriptive analysis

In [71]:
combined_df.to_csv(os.path.join(output_path, 'combined_dataframe.csv'), index=False)

In [50]:
combined_df.head(3)

,analysis_id,experiment_type,pipeline_version,instrument_platform,study_id,sample_id,assembly_run_id,study_name,n_samples,bioproject,centre_name,biomes
0,MGYA00166416,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488844,ERR2586218,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
1,MGYA00166417,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488846,ERR2586220,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
2,MGYA00166418,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488842,ERR2586216,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge


In [68]:
combined_df.dtypes

analysis_id             object
experiment_type         object
pipeline_version       float64
instrument_platform     object
study_id                object
sample_id               object
assembly_run_id         object
study_name              object
n_samples                int64
bioproject              object
centre_name             object
biomes                  object
dtype: object

In [99]:
def explore_dataset(dataset):
    """_summary_

    Args:
        dataset (_type_): _description_
    """

    # count number of unique 'study_id'
    print(f"\nTotal number of unique studies: {dataset["study_id"].nunique()}")

    print(f"\nNumber of unique assembly_run_id per study_id:{combined_df.groupby("study_id")["assembly_run_id"].nunique()}")

    # missing values
    print(f"\nMissing values per variable:{dataset.isnull().sum()}")
    
    print(f"\nNumber of samples per biome:{combined_df.groupby('biomes')['n_samples'].median().reset_index()}")

In [100]:
explore_dataset(combined_df)


Total number of unique studies: 117

Number of unique assembly_run_id per study_id:study_id
MGYS00000423      1
MGYS00000425      1
MGYS00000555      1
MGYS00000597     16
MGYS00000606      1
               ... 
MGYS00005614      8
MGYS00005769     11
MGYS00005802      6
MGYS00005846    110
MGYS00006570    152
Name: assembly_run_id, Length: 117, dtype: int64

Missing values per variable:analysis_id            0
experiment_type        0
pipeline_version       0
instrument_platform    0
study_id               0
sample_id              0
assembly_run_id        0
study_name             0
n_samples              0
bioproject             0
centre_name            0
biomes                 0
dtype: int64

Number of samples per biome:                                              biomes  n_samples
0                         root:Engineered:Wastewater       81.0
1        root:Engineered:Wastewater:Activated Sludge       12.0
2  root:Engineered:Wastewater:Activated Sludge, r...       70.0
3   root:En

In [51]:
combined_df.shape

(1614, 12)

In [52]:
combined_df.experiment_type.unique()

array(['metagenomic', 'metatranscriptomic', 'assembly'], dtype=object)

In [56]:
# 4. Esplorare la distribuzione di 'experiment_type' e 'biomes'
experiment_type_counts = combined_df["experiment_type"].value_counts()
biomes_counts = combined_df["biomes"].value_counts()
print("\nDistribuzione di experiment_type:")
print(experiment_type_counts)
print("\nDistribuzione di biomes:")
print(biomes_counts)


Distribuzione di experiment_type:
experiment_type
metagenomic           985
assembly              530
metatranscriptomic     99
Name: count, dtype: int64

Distribuzione di biomes:
biomes
root:Engineered:Wastewater:Water and sludge                                                      708
root:Engineered:Wastewater                                                                       497
root:Engineered:Wastewater:Activated Sludge                                                      225
root:Engineered:Wastewater:Activated Sludge, root:Engineered:Wastewater:Industrial wastewater     70
root:Engineered:Wastewater:Industrial wastewater                                                  67
root:Engineered:Wastewater:Industrial wastewater:Petrochemical                                    24
root:Engineered:Wastewater:Nutrient removal:Dissolved organics (anaerobic)                        12
root:Engineered:Wastewater:Nutrient removal:Biological phosphorus removal:Activated sludge         8
root

In [57]:
# missing data
any_missing_data = combined_df.isnull().values.any()

print(f"Are there any missing data in the dataframe? {'yes' if any_missing_data else 'no'}")

Are there any missing data in the dataframe? no


In [63]:
combined_df.head()

,analysis_id,experiment_type,pipeline_version,instrument_platform,study_id,sample_id,assembly_run_id,study_name,n_samples,bioproject,centre_name,biomes
0,MGYA00166416,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488844,ERR2586218,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
1,MGYA00166417,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488846,ERR2586220,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
2,MGYA00166418,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488842,ERR2586216,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
3,MGYA00166419,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488841,ERR2586215,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge
4,MGYA00166420,metagenomic,4.1,Illumina HiSeq 2500,MGYS00002383,ERS2488845,ERR2586219,Antibiotic manufacturing effluent enriches res...,6,PRJEB26809,UNIVERSITY OF GOTHENBURG,root:Engineered:Wastewater:Activated Sludge


In [64]:
combined_df.experiment_type.unique()

array(['metagenomic', 'metatranscriptomic', 'assembly'], dtype=object)

In [98]:
experiment_type_counts = combined_df["experiment_type"].value_counts()
print(experiment_type_counts)

experiment_type
metagenomic           985
assembly              530
metatranscriptomic     99
Name: count, dtype: int64


In [89]:
combined_df.dtypes


analysis_id             object
experiment_type         object
pipeline_version       float64
instrument_platform     object
study_id                object
sample_id               object
assembly_run_id         object
study_name              object
n_samples                int64
bioproject              object
centre_name             object
biomes                  object
dtype: object

In [97]:
combined_df.shape

(1614, 12)

In [96]:
### do we have different pipeline versions considering different IDs?

analysis_versions = combined_df.groupby('analysis_id')['pipeline_version'].nunique()
study_versions = combined_df.groupby('study_id')['pipeline_version'].nunique()
sample_versions = combined_df.groupby('sample_id')['pipeline_version'].nunique()
run_versions = combined_df.groupby('assembly_run_id')['pipeline_version'].nunique()

multiple_versions_ana = analysis_versions[analysis_versions > 1]
multiple_versions_stu = study_versions[study_versions > 1]
multiple_versions_sam = sample_versions[sample_versions > 1]
multiple_versions_run = run_versions[run_versions > 1]

print(len(multiple_versions_ana))
print(len(multiple_versions_stu))
print(len(multiple_versions_sam))
print(len(multiple_versions_run))

0
4
162
197


In [92]:
combined_df[combined_df['assembly_run_id'] == 'ERR1713331']

,analysis_id,experiment_type,pipeline_version,instrument_platform,study_id,sample_id,assembly_run_id,study_name,n_samples,bioproject,centre_name,biomes
357,MGYA00216627,metagenomic,4.1,Illumina HiSeq 3000,MGYS00001312,ERS1426778,ERR1713331,Global surveillance of infectious diseases and...,179,PRJEB13831,DTU-GE,root:Engineered:Wastewater:Water and sludge
533,MGYA00085654,metagenomic,3.0,Illumina HiSeq 3000,MGYS00001312,ERS1426778,ERR1713331,Global surveillance of infectious diseases and...,179,PRJEB13831,DTU-GE,root:Engineered:Wastewater:Water and sludge


## Prossimi steps

1. creare una nuova variabile chiamata "name_id" in cui si considerano solo le prime 3 lettere e vengono printate nella nuova colonna.
2. unire study_id, sample_id, assembly_run_id, n_samples e bioproject in un unica variabile stringa e se doppione considerare solo quella con versione piu alta; per fare cio forse conviene utilizzare un dizionario per convertire i float 
3. creare una nuova variabile in grado di separare le prime 3 lettere di assembly_run_id
4. creare funzione in grado di scaricare ERR da un sito e altri nominativi da un altro e salvarli in un file txt
5. Possiamo creare i file json al di fuori del ciclo for? in questo modo posso creare solo due json e non avere il problema della sovrascrizione.
6. adattare la funzione al file finale per scaricare i dati fastq

10. scrivere l'analisi in file main.py and functions.py